# 载入MNIST数据集

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
sess = tf.InteractiveSession()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# 展示MNIST数据集图片

![avatar](https://images2017.cnblogs.com/blog/1005218/201711/1005218-20171108192955528-1822463409.png)

# 权重和偏置的初始化函数

In [3]:
def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 输入和标签

In [4]:
x = tf.placeholder(tf.float32, [None,784])
y_ = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1,28,28,1])

# 第一个卷积层

In [5]:
W_conv1 = weight_varible([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1,1,1,1], padding='SAME') + b_conv1)
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# 第二个卷积层

In [6]:
W_conv2 = weight_varible([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1,1,1,1], padding='SAME') + b_conv2)
h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# 全连接层

In [7]:
W_fc1 = weight_varible([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

# 输出层

In [8]:
W_fc2 = weight_varible([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

# 使用交叉熵作为损失函数

In [9]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y_conv),reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 定义评测准确率的操作

In [10]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 训练

In [11]:
train = False
saver=tf.train.Saver()

tf.global_variables_initializer().run()

if train:
    for i in range(10000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1]})
            print("step %d, taining accuracy %g" %(i, train_accuracy))
        train_step.run(feed_dict={x:batch[0],y_:batch[1]})
    saver.save(sess,'ckpt/mnist.ckpt',global_step=i)
else:
    model_file=tf.train.latest_checkpoint('ckpt/')
    saver.restore(sess,model_file)

INFO:tensorflow:Restoring parameters from ckpt/mnist.ckpt-9999


# 训练完成之后，在最终的测试集上进行全面的测试

In [12]:
print("test accuracy %g" %accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

test accuracy 0.9838


# 在测试集中随机挑选图片进行识别

In [13]:
import matplotlib.pyplot as plt
import numpy as np

y_predict = tf.argmax(y_conv, 1)

index = 1231

plt.matshow(np.reshape(mnist.test.images[index,:], (28,28)), cmap=plt.get_cmap('gray'))

print('Predict label: ',y_predict.eval(feed_dict={x:np.reshape(mnist.test.images[index,:], (-1,784))})[0])
print('True label: ', np.argmax(mnist.test.labels[index]))
